# Trying to use pocketsphinx to word align

> "Because timing accuracy in ASR is getting progressively worse, look backwards"

- branch: master
- comments: false
- categories: [pocketsphinx, hsi, alignment]

In [1]:
from pydub import AudioSegment

In [14]:
%%capture
%pip install pocketsphinx

In [31]:
MAPPING = """
ɑː AA
æ AE
ə AH
ɐ AH
ʌ AH
ɔː AO
aʊ AW
aɪ AY
b B
tʃ CH
d D
ð DH
ɛ EH
ɚ ER
ɜː ER
eɪ EY
f F
ɡ G
h HH
ɪ IH
i IY
iː IY
dʒ JH
k K
l L
m M
n N
ŋ NG
oʊ OW
ɔɪ OY
p P
ɹ R
s S
ʃ SH
t T
θ TH
ʊ UH
uː UW
v V
w W
j Y
z Z
ʒ ZH
ɾ D
"""

In [62]:
espeak_to_cmudict = {}
for line in MAPPING.split("\n"):
    if line == "":
        continue
    line = line.strip()
    parts = line.split(" ")

    if len(parts) != 2:
        print(line)
        continue
    k, v = line.split(" ")
    if not k in espeak_to_cmudict:
        espeak_to_cmudict[k] = v
    

In [81]:
import re

cmudict_keys = espeak_to_cmudict.keys()
cmudict_keys = sorted(cmudict_keys, key=len, reverse=True)
espeak_regex = re.compile(rf"({'|'.join(cmudict_keys)})")

def cmudictify(espeak):
    espeak = espeak.replace("ˈ", "").replace("ˌ", "")
    return " ".join([espeak_to_cmudict[x] for x in re.findall(espeak_regex, espeak)])

In [58]:
EGTEXT = "Yeah, that's true. I mean, they are the same size and they are a little bit, but I think I I should go more for something that style."
EGPHON = "/jˈæ ðˈæs tɹˈuː ə mˈiːn ðˈeɪ ɚ ðə sˈeɪm sˈaɪz ən ðˈeɪ ˈɑːɹ ə lˈɪɾə bˈɪɾ bˈʌt ˈaɪ θˈɪŋk ˈaɪ ˈaɪ ʃˈʊ ɡˈoʊ mˈɔːɹ fɚ sˈʌmθɪŋ ðˈæt stˈaɪl./"
EGFILE = "/Users/joregan/Playing/hsi/audio/hsi_7_0719_210_001_main.wav"
EGSTART = 70.028
EGEND = 75.441

In [79]:
def normword(text):
    text = text.strip(",.;:!?")
    return text.lower()

def normphon(phon):
    phon = phon.strip(",.;:!?")
    return phon

def make_lexicon(text, phon):
    if phon.startswith("/") and phon.endswith("/"):
        phon = phon[1:-1]
    words = [normword(x) for x in text.split(" ")]
    phonwords = [cmudictify(normphon(x)) for x in phon.split(" ")]
    assert len(words) == len(phonwords)
    output = list(set(zip(words, phonwords)))
    return output
    

In [83]:
lex = make_lexicon(EGTEXT, EGPHON)

In [141]:
audio = AudioSegment.from_file(EGFILE)

In [142]:
audio = audio.set_frame_rate(16000)

In [143]:
seg = audio[int(EGSTART * 1000):int(EGEND * 1000)]

In [128]:
def make_ps_dict(entries):
    counts = {}
    output = []
    lex = sorted(entries)
    for entry in lex:
        count = 1
        if not entry[0] in counts:
            counts[entry[0]] = 1
        else:
            counts[entry[0]] += 1
            count = counts[entry[0]]
        if count != 1:
            subscript = f"({count})"
        else:
            subscript = ""
        output.append(f"{entry[0]}{subscript} {entry[1]}")
    return output

In [97]:
def make_fsg_transitions_from_text(text):
    words = [normword(x) for x in text.split(" ")]
    enum = [x for x in enumerate(words)]
    trans = [(x[0], x[0] + 1, 1.0, x[1]) for x in enum]
    return trans

In [99]:
fsgt = make_fsg_transitions_from_text(EGTEXT)

In [100]:
start_state = fsgt[0][0]
end_state = fsgt[-1][1]

In [160]:
import pocketsphinx

This was the first attempt. Adding `None` for the dictionary (as the docs suggested) didn't help: can't add words to a dictionary that doesn't exist.

This may yet be a thing, because I can't be sure that it really failed to align using the grammar: audio handling sucks, and I should maybe have passed ffmpeg parameters before writing the audio.

In [161]:
import tempfile
entries = make_ps_dict(lex)

with (
    tempfile.NamedTemporaryFile(suffix=".dict") as dictf,
    tempfile.NamedTemporaryFile(suffix=".wav") as wavf,
):
    with open(dictf.name, "w") as dictout:
        for entry in entries:
            dictout.write(entry + "\n")
    
    seg.export(wavf.name, format="wav")

    decoder = pocketsphinx.Decoder(lm=None, dict=dictf.name)
    fsg = decoder.create_fsg("dummy", start_state, end_state, fsgt)
    decoder.add_fsg("dummy", fsg)
    decoder.activate_search("dummy")
    decoder.start_utt()
    # decoder.process_raw(seg.get_array_of_samples('B'))
    decoder.process_raw(wavf.read(), full_utt=True)
    decoder.end_utt()

In [166]:
decoder.seg()

ERROR: "fsg_search.c", line 944: Final result does not match the grammar in frame 1082


In [164]:
fsg.writefile("/tmp/fsm")

In [165]:
!cat /tmp/fsm

FSG_BEGIN dummy
NUM_STATES 29
START_STATE 0
FINAL_STATE 28
TRANSITION 0 0 0.000000 [NOISE]
TRANSITION 0 0 0.005001 <sil>
TRANSITION 0 1 1.000000 yeah
TRANSITION 1 1 0.000000 [NOISE]
TRANSITION 1 1 0.005001 <sil>
TRANSITION 1 2 1.000000 that's
TRANSITION 2 2 0.000000 [NOISE]
TRANSITION 2 2 0.005001 <sil>
TRANSITION 2 3 1.000000 true
TRANSITION 3 3 0.000000 [NOISE]
TRANSITION 3 3 0.005001 <sil>
TRANSITION 3 4 1.000000 i(2)
TRANSITION 3 4 1.000000 i
TRANSITION 4 4 0.000000 [NOISE]
TRANSITION 4 4 0.005001 <sil>
TRANSITION 4 5 1.000000 mean
TRANSITION 5 5 0.000000 [NOISE]
TRANSITION 5 5 0.005001 <sil>
TRANSITION 5 6 1.000000 they
TRANSITION 6 6 0.000000 [NOISE]
TRANSITION 6 6 0.005001 <sil>
TRANSITION 6 7 1.000000 are(2)
TRANSITION 6 7 1.000000 are
TRANSITION 7 7 0.000000 [NOISE]
TRANSITION 7 7 0.005001 <sil>
TRANSITION 7 8 1.000000 the
TRANSITION 8 8 0.000000 [NOISE]
TRANSITION 8 8 0.005001 <sil>
TRANSITION 8 9 1.000000 same
TRANSITION 9 9 0.000000 [NOISE]
TRANSITION 9 9 0.005001 <sil>
TRA

In [167]:
with open("/tmp/mytmp.dict", "w") as dictout:
    for entry in entries:
        dictout.write(entry + "\n")


In [168]:
seg.export("/tmp/clip.wav", format="wav")

<_io.BufferedRandom name='/tmp/clip.wav'>

In [171]:
!sox /tmp/clip.wav $(pocketsphinx soxflags) > /tmp/ps.raw
psjson=!pocketsphinx align /tmp/ps.raw "yeah that's true i mean they are the same size and they are a little bit but i think i i should go more for something that style"

In [174]:
psjson

['{"b":0.000,"d":5.410,"p":1.000,"t":"yeah that\'s true i mean they are the same size and they are a little bit but i think i i should go more for something that style","w":[{"b":0.000,"d":0.250,"p":0.964,"t":"yeah"},{"b":0.250,"d":0.150,"p":0.937,"t":"that\'s"},{"b":0.400,"d":0.200,"p":0.978,"t":"true"},{"b":0.600,"d":0.060,"p":0.974,"t":"i"},{"b":0.660,"d":0.180,"p":0.980,"t":"mean"},{"b":0.840,"d":0.140,"p":0.979,"t":"they"},{"b":0.980,"d":0.070,"p":0.983,"t":"are(2)"},{"b":1.050,"d":0.110,"p":0.987,"t":"the"},{"b":1.160,"d":0.260,"p":0.974,"t":"same"},{"b":1.420,"d":0.380,"p":0.956,"t":"size"},{"b":1.800,"d":0.120,"p":0.975,"t":"and"},{"b":1.920,"d":0.110,"p":0.986,"t":"they"},{"b":2.030,"d":0.140,"p":0.983,"t":"are(2)"},{"b":2.170,"d":0.030,"p":0.990,"t":"a"},{"b":2.200,"d":0.180,"p":0.914,"t":"little"},{"b":2.380,"d":0.140,"p":0.980,"t":"bit"},{"b":2.520,"d":0.220,"p":0.935,"t":"but"},{"b":2.740,"d":0.110,"p":0.983,"t":"i"},{"b":2.850,"d":0.260,"p":0.968,"t":"think"},{"b":3.110,"

In [175]:
import json
data = json.loads(str(psjson[0]))

In [176]:
data

{'b': 0.0,
 'd': 5.41,
 'p': 1.0,
 't': "yeah that's true i mean they are the same size and they are a little bit but i think i i should go more for something that style",
 'w': [{'b': 0.0, 'd': 0.25, 'p': 0.964, 't': 'yeah'},
  {'b': 0.25, 'd': 0.15, 'p': 0.937, 't': "that's"},
  {'b': 0.4, 'd': 0.2, 'p': 0.978, 't': 'true'},
  {'b': 0.6, 'd': 0.06, 'p': 0.974, 't': 'i'},
  {'b': 0.66, 'd': 0.18, 'p': 0.98, 't': 'mean'},
  {'b': 0.84, 'd': 0.14, 'p': 0.979, 't': 'they'},
  {'b': 0.98, 'd': 0.07, 'p': 0.983, 't': 'are(2)'},
  {'b': 1.05, 'd': 0.11, 'p': 0.987, 't': 'the'},
  {'b': 1.16, 'd': 0.26, 'p': 0.974, 't': 'same'},
  {'b': 1.42, 'd': 0.38, 'p': 0.956, 't': 'size'},
  {'b': 1.8, 'd': 0.12, 'p': 0.975, 't': 'and'},
  {'b': 1.92, 'd': 0.11, 'p': 0.986, 't': 'they'},
  {'b': 2.03, 'd': 0.14, 'p': 0.983, 't': 'are(2)'},
  {'b': 2.17, 'd': 0.03, 'p': 0.99, 't': 'a'},
  {'b': 2.2, 'd': 0.18, 'p': 0.914, 't': 'little'},
  {'b': 2.38, 'd': 0.14, 'p': 0.98, 't': 'bit'},
  {'b': 2.52, 'd'

In [179]:
from praatio import textgrid

In [178]:
for word in data["w"]:
    print(word)

{'b': 0.0, 'd': 0.25, 'p': 0.964, 't': 'yeah'}
{'b': 0.25, 'd': 0.15, 'p': 0.937, 't': "that's"}
{'b': 0.4, 'd': 0.2, 'p': 0.978, 't': 'true'}
{'b': 0.6, 'd': 0.06, 'p': 0.974, 't': 'i'}
{'b': 0.66, 'd': 0.18, 'p': 0.98, 't': 'mean'}
{'b': 0.84, 'd': 0.14, 'p': 0.979, 't': 'they'}
{'b': 0.98, 'd': 0.07, 'p': 0.983, 't': 'are(2)'}
{'b': 1.05, 'd': 0.11, 'p': 0.987, 't': 'the'}
{'b': 1.16, 'd': 0.26, 'p': 0.974, 't': 'same'}
{'b': 1.42, 'd': 0.38, 'p': 0.956, 't': 'size'}
{'b': 1.8, 'd': 0.12, 'p': 0.975, 't': 'and'}
{'b': 1.92, 'd': 0.11, 'p': 0.986, 't': 'they'}
{'b': 2.03, 'd': 0.14, 'p': 0.983, 't': 'are(2)'}
{'b': 2.17, 'd': 0.03, 'p': 0.99, 't': 'a'}
{'b': 2.2, 'd': 0.18, 'p': 0.914, 't': 'little'}
{'b': 2.38, 'd': 0.14, 'p': 0.98, 't': 'bit'}
{'b': 2.52, 'd': 0.22, 'p': 0.935, 't': 'but'}
{'b': 2.74, 'd': 0.11, 'p': 0.983, 't': 'i'}
{'b': 2.85, 'd': 0.26, 'p': 0.968, 't': 'think'}
{'b': 3.11, 'd': 0.19, 'p': 0.975, 't': 'i'}
{'b': 3.3, 'd': 0.09, 'p': 0.977, 't': 'i'}
{'b': 3.39, 